## Tutoriel : interagir avec le système de stockage S3 du SSP Cloud (MinIO)

In [14]:
import os
import sys

import pandas as pd
import numpy as np
import s3fs
import zipfile

sys.path.insert(0, 'datasets/')
import dataset
import roomsetup
import treatedroom

%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Récupérer les données d'un challenge

In [3]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [5]:
# Télécharger les données dans le service
PATH_IN = 'samicarret/diffusion/TreatedRoomSmallSet2.zip'
fs.download(PATH_IN, 'data/HC3.zip')

[None]

In [6]:
# Décompresser les données
with zipfile.ZipFile("data/HC3.zip","r") as zip_file:
    zip_file.extractall("data/")

### Lecture des fichiers

In [10]:
file = np.load('data/TreatedRoomSmallSet/Human1/music_audio.npy')

In [11]:
print(file)
file.shape

[[[  0  -3  -1 ...  -4  -6  -5]
  [ -9  -9  -8 ...   3   3   3]
  [  5   5   4 ...  -7  -8  -7]
  ...
  [  0   0   1 ...   5   5   3]
  [  5   5   3 ...   6   6   4]
  [ 49  46  47 ...  -8  -7  -8]]

 [[ 22  21  23 ...  -8 -12  -9]
  [  7   8   8 ...  -1  -3  -5]
  [ 13  11  11 ... -10 -11 -11]
  ...
  [  4   4   4 ... -15 -13 -13]
  [  7   3   2 ... -16 -17 -18]
  [-12 -10 -13 ... -15 -12 -17]]

 [[  5   5   6 ...  21  20  20]
  [  1   2   0 ...   5   4   2]
  [ 11  10  15 ...  17  19  21]
  ...
  [ -1  -2  -2 ...  24  24  23]
  [ -3  -4  -6 ...  12  12  13]
  [ 15  16  18 ...  33  35  35]]]


(3, 10, 672000)